In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer, QuantumCircuit
from qiskit.opflow import Z, I, StateFn
from qiskit.utils import QuantumInstance
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap, ZFeatureMap, PauliFeatureMap
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, ADAM, SPSA


import pandas as pd

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from qiskit_machine_learning.algorithms.objective_functions import OneHotObjectiveFunction
from qiskit_machine_learning.neural_networks import TwoLayerQNN, CircuitQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR

from typing import Union

from qiskit_machine_learning.exceptions import QiskitMachineLearningError

from IPython.display import clear_output

from sklearn.feature_extraction import DictVectorizer

In [2]:
from qiskit import IBMQ

# IBMQ.save_account(TOKEN)
IBMQ.enable_account(
    "PUT_IBMQ_TOKEN_HERE", )
IBMQ.providers()    # List all available providers

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_m

In [4]:
backend = provider.get_backend('ibmq_lima')
backend

<IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns 
    specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(
        vec.fit_transform(df[cols].to_dict('records')) \
        .toarray()
    )
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df


In [6]:
quantum_instance = QuantumInstance(backend, shots=1024)

In [7]:
objective_func_vals = []
weights_array = []
def callback_graph(weights, obj_func_eval):
    objective_func_vals.append(obj_func_eval)
    weights_array.append(weights)

In [8]:
RANDOM_STATE = 37
num_inputs = 4 # number of qubits
class_labels = ['Iris-setosa', 'Iris-versicolor','Iris-virginica']

In [9]:
def normalizeData(DATA_PATH = "iris.data"):
    """
    Normalizes the data
    """
    # Reads the data
    data = pd.read_csv(DATA_PATH)
    print(data)
    data = shuffle(data, random_state=RANDOM_STATE)
    
    data = encode_onehot(data, cols=['class'])
    print(data)
    # Split into data and result
    parameters = [
        'sepal_length', 
        'sepal_width', 
        'petal_length', 
        'petal_width'
    ]
    classes = [
        'class=Iris-setosa', 
        'class=Iris-versicolor', 
        'class=Iris-virginica'
    ]
    X, Y = data[parameters].values, data[classes].values
    
    print(X)
    print(Y)
    
    # normalize the data
    X = normalize(X)
    split = train_test_split(
        X, 
        Y, 
        test_size=0.3, 
        random_state=RANDOM_STATE
    )
    X_train, X_test, Y_train, Y_test = split
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = normalizeData()

     sepal_length  sepal_width  petal_length  petal_width           class
0             5.1          3.5           1.4          0.2     Iris-setosa
1             4.9          3.0           1.4          0.2     Iris-setosa
2             4.7          3.2           1.3          0.2     Iris-setosa
3             4.6          3.1           1.5          0.2     Iris-setosa
4             5.0          3.6           1.4          0.2     Iris-setosa
..            ...          ...           ...          ...             ...
145           6.7          3.0           5.2          2.3  Iris-virginica
146           6.3          2.5           5.0          1.9  Iris-virginica
147           6.5          3.0           5.2          2.0  Iris-virginica
148           6.2          3.4           5.4          2.3  Iris-virginica
149           5.9          3.0           5.1          1.8  Iris-virginica

[150 rows x 5 columns]
     sepal_length  sepal_width  petal_length  petal_width  class=Iris-setosa  \
74      

/Library/Python/3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
output_shape = len(class_labels)

def interpretation(x):
    return f"{x:b}".count("1") % output_shape

def construct_circuit(
    feature_map, 
    ansatz, 
    quantum_instance, 
    number_of_inputs
):
    qc = QuantumCircuit(number_of_inputs)
    qc.append(feature_map, range(number_of_inputs))
    qc.append(ansatz, range(number_of_inputs))
    
    return CircuitQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        sampling=True,
        interpret=interpretation,
        quantum_instance=quantum_instance)

In [11]:
feature_map = ZFeatureMap(num_inputs, reps=2)
ansatz = RealAmplitudes(num_inputs, reps=1)

In [12]:
circuit_qnn = construct_circuit(feature_map, ansatz, quantum_instance, num_inputs)

In [ ]:
def compare_elements(first, second):
    return first == second

vcompare_elements = np.vectorize(compare_elements)

def is_all_true(arg):
    newarr = []
    for a in arg:
        newarr.append(np.all(a == True))
    return newarr

def score_model(observed, actual_values):
    array_is_correct = is_all_true(
        vcompare_elements(observed, actual_values)
    )
    number_of_correct = array_is_correct.count(True)
    return number_of_correct / len(array_is_correct)

In [ ]:
def run(circuit, X, Y, weights):
    out = circuit.forward(X, weights).astype(int)
    result = []
    for value in out:
        _, counts = np.unique(value, return_counts=True)
        ind = np.argmax(counts)
        if ind == 2:
            result.append([0,0,1])
        if ind == 1:
            result.append([0,1,0])
        if ind == 0:
            result.append([1,0,0])
    return result

def run_and_score(circuit, X, Y, weights):
    result = run(circuit, X, Y, weights)
    return score_model(result, Y)

# test_and_score(circuit_qnn, X_test, Y_test, weights_array[-1])

In [18]:
test_cases = [
#     ("COBYLA200-ZFeature4-RealAmplitudes4", 
#      ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), 
#      [1.96652569,  0.24217319,  1.47837951,  0.83915742,  2.29978635,
#         0.69589474,  0.24225658,  0.09114969,  0.6943839 ,  1.70546927,
#         0.0490381 ,  1.89544691,  0.85960574,  1.35149849, -0.3217444 ,
#        -0.31325965,  0.39365852,  0.60165538,  0.70124588,  0.53969418]),
#     ("COBYLA200-ZZFeature1-RealAmplitudes2", ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=2),
#     [0.78063678, 2.0224423 , 1.92216082, 0.77181018, 0.33547541,
#        0.28029247, 2.72057156, 1.16207983, 0.28140304, 0.29973641,
#        1.10931136, 0.21435968]),
#     ("COBYLA200-ZZFeature2-RealAmplitudes2", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2),
#      [ 0.64819338,  1.78587224,  1.29973788,  0.90789126, -0.77930814,
#        -0.92923439, -0.62550483,  1.14443345,  1.22306553,  0.58302753,
#         1.69812501,  0.96229817]),
#     ("COBYLA200-ZZFeature1-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4),
#      [ 0.82100403,  0.70563947,  0.75850099,  0.43672643,  0.95479829,
#         0.5186458 ,  0.80051422,  1.03009911,  2.25575419, -0.31275686,
#         1.0060153 ,  0.82027366,  0.4635406 ,  0.50547487,  0.27642783,
#         1.0440678 , -0.36731861, -0.33422435, -0.46824865,  0.25620795]),
#     ("COBYLA200-ZZFeature2-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4),
#      [ 0.4911561 ,  0.31609578, -0.38162514,  1.15827159, -0.29625443,
#         0.15749164,  2.13539031,  1.16861619,  1.59499743,  1.00445452,
#         1.05213066,  0.13898716,  0.51715947,  1.55181602,  1.6747511 ,
#         0.43560374,  0.66306244, -0.03375648,  0.09613641,  1.66362037]),
#     ("SPSA200-ZFeature4-RealAmplitudes2", ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2),
#     [ 0.48447912,  0.85423841,  1.23879561,  0.87086215,  0.93328164,
#        -0.12087993,  0.43203537,  0.28788571,  1.41622501,  0.95067168,
#         0.30541062,  1.34815698]),
#     ("SPSA200-ZFeature4-RealAmplitudes4", ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4),
#     [ 1.46897148,  1.61612961,  0.66400888, -1.38317863, -1.18515629,
#         1.39184363, -0.02823102,  1.22893046,  1.78280019,  0.02352316,
#         1.84723413, -0.07711311,  0.26445542, -0.15511904,  0.32122471,
#         0.69553548, -1.73949634,  2.08829505, -0.6182551 , -0.3715629 ]),
#     ("SPSA200-ZZFeature2-RealAmplitudes1", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1),
#     [ 0.12544794,  1.34447698, -1.56045246,  0.33478607, -0.21053891,
#         1.8375332 ,  0.95011212,  2.87425482]),
#     ("SPSA200-ZZFeature4-RealAmplitudes1", ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1),
#     [-1.42218514,  1.80886525,  0.02010241,  0.82039011, -0.15961447,
#        -1.24580342,  0.184429  ,  1.99394651]),
#     ("SPSA200-ZZFeature1-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4),
#     [-0.03531448,  0.95047674, -0.29809365,  2.08363356,  0.02681253,
#         1.50036644, -0.48563553, -1.13522444,  0.22765716, -0.53121227,
#         0.03809585, -0.47580457,  1.25593285,  0.28747781,  0.66827889,
#        -1.20637956, -0.6382    ,  2.31508532,  0.68727489, -1.19170088]),
#     ("SPSA200-ZZFeature2-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4),
#      [ 8.57568228e-01, -1.47517684e+00, -6.36786069e-01,  4.10163315e-01,
#         1.25928106e+00,  8.46976053e-01,  1.64939020e-03,  3.83659638e-01,
#         1.14938131e+00,  9.61452264e-01, -6.84435098e-01,  2.27907110e+00,
#         6.58920889e-01,  1.15749164e+00,  4.84793493e-01, -1.27893428e-02,
#         9.13764668e-01,  8.64911178e-01, -3.23748533e-01,  1.76562156e+00]),
#     ("SPSA200-ZZFeature4-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4),
#     [ 1.89465409,  1.57469329, -0.14706226,  2.13174852,  1.91503964,
#         0.05349635,  1.57194648,  1.97840214, -0.00579625,  0.09230878,
#         0.49631138,  2.5894188 ,  0.73304498,  0.80060568,  0.24989982,
#         3.90697461,  1.5261834 , -2.39113564, -0.46044484,  1.64944285]),
#     ("SLSQP200-ZFeature4-RealAmplitudes4", ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4),
#     [ 0.52472758, -5.33081038,  0.64436542,  1.2462468 ,  0.66514955,
#         1.82783489,  5.85190487, -3.14194131,  6.39873267, -3.18595594,
#         5.3332733 ,  0.58693821, -3.39888723, -2.69845055, -1.41990395,
#         3.88865357, -1.15479753, -0.27437946,  4.63664874,  5.17957124]),
#     ("SLSQP200-ZFeature2-RealAmplitudes1", ZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1),
#     [0.31894041, 0.89611847, 1.09508117, 1.55556388, 0.15318628,
#        0.14088195, 0.62011282, 1.82333566])
#     ("SLSQP200-ZZFeature4-RealAmplitudes1", ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=1),
#     [-0.50834331, -0.51311476,  1.976053  ,  7.23649327, -1.69539646,
#         4.99074994,  7.31828862,  6.0519525 ]
#     ),
#     ("SLSQP200-ZZFeature2-RealAmplitudes2", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=2),
#      [ 0.97388011,  2.0565755 ,  1.2692307 , -0.49420958,  0.4929385 ,
#        -0.60743675, -1.93669273, -0.54927114,  0.61931404,  1.30669587,
#         1.79120397,  0.31871543]),
#     ("SLSQP200-ZZFeature4-RealAmplitudes2", ZZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2),
#     [ 1.63359876, -0.32988454, -0.60956253,  4.96152179, -2.33766433,
#         1.00834894,  1.74299519,  0.99391834, -0.9879509 , -2.03564731,
#        -0.67112109,  1.03631379]),
#     ("SLSQP200-ZZFeature2-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4),
#     [-0.52573993,  0.65528673, -0.35367426,  0.82543965, -0.44807967,
#        -1.49843549, -0.24870379,  0.57662176, -0.82378355, -1.77832255,
#         0.44396715,  2.54303275,  1.15362686,  3.59116526,  2.16540144,
#         1.53343009, -2.54384715,  1.63921725,  0.43171186,  4.43870397]),
#     ("SLSQP200-ZZFeature4-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=4),
#     [ 3.31613777,  6.06667742, -3.95316258, -1.69295251,  0.19167912,
#        -2.62664836,  2.83295204, -1.33366066, -1.03547179,  3.82680006,
#        -6.81343191,  3.67560685,  1.6656707 , -6.57166221,  0.31083883,
#         4.17857274,  0.61017123,  7.36818046, -0.8611246 , -2.95814297])
#     ("SLSQP200-PauliFeature2-RealAmplitudes1", PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1),
#      [ 3.11676948, -1.26906072, -0.70324747,  3.60346914, -1.63542024,
#        -1.44592427, -0.16103554, -1.94785953]),
#     ("SLSQP200-PauliFeature1-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2),
#     [ 0.44331459,  0.56736968,  0.98057143,  2.26051744, -0.02899645,
#         1.03850501, -0.21954115,  1.82585364,  1.53487442, -0.50782958,
#         0.44577786,  0.57430938]),
#     ("SLSQP200-PauliFeature1-RealAmplitudes4", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4),
#     [-2.76971833, -3.52263685,  1.05064507, -1.37907321, -1.30840688,
#        -1.38316854,  1.61945575, -2.38849097,  1.76322422, -1.19731837,
#         0.31281012,  0.18219535,  7.21919779, -1.53670491,  0.13906521,
#         4.12032984, -6.73180806, -0.42247248, -0.1367392 ,  1.73066456]),
#     ("COBYLA200-PauliFeature4-RealAmplitudes1", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1),
#     [ 0.14355749,  2.47275515,  1.97896004, -0.69595873, -0.25622802,
#         0.74147221,  0.4871683 ,  0.53272661]),
#     ("COBYLA200-PauliFeature1-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2),
#     [ 1.2406585 ,  0.71026949,  0.64973388, -0.82795603,  0.22886487,
#         1.67095024,  1.49102572,  2.2561861 ,  0.93238461,  1.68436845,
#         0.84875139, -0.60315579]),
#     ("COBYLA200-PauliFeature2-RealAmplitudes4", PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4),
#     [-0.3335182 , -0.12115916,  1.98050369,  1.80022474, -0.73744801,
#        -0.21870494,  0.64690349,  0.6411746 ,  1.66614741,  1.84256641,
#         1.0567516 , -0.35900059,  1.63904842,  1.4157956 ,  0.6132988 ,
#         0.17818834,  1.13889294,  1.55545486,  0.76825807,  1.30665367]),
#     ("COBYLA200-PauliFeature4-RealAmplitudes4", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4),
#     [ 0.60302796,  1.89898044, -0.14106747,  1.83477869,  1.19765699,
#         0.66237451, -0.02829689,  1.351477  ,  1.28607333,  0.8597519 ,
#         1.75212796,  0.30145751,  1.25550447,  0.83454254, -0.16654966,
#         1.3411416 , -0.1700231 ,  1.73733933,  1.70572365,  0.2803679 ]),
#     ("SPSA200-PauliFeature1-RealAmplitudes1", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1),
#      [ 1.59238438,  0.92611349, -0.17279913, -0.7114918 ,  1.21036182,
#         1.45891369, -0.41394514,  1.91146358]),
#     ("SPSA200-PauliFeature4-RealAmplitudes1", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1),
#     [-0.06500449,  2.17072138,  2.22459104, -1.05176998, -0.30855382,
#         0.60190422,  0.62994114,  0.57937807]),
#     ("SPSA200-PauliFeature4-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2),
#     [ 0.70445125,  1.25676107,  0.73025549,  0.56406546, -0.05183136,
#         0.4902514 ,  1.53389059, -0.19832704,  0.20497459,  0.88999083,
#         1.60772945, -0.35216852]),
#     ("SPSA200-PauliFeature1-RealAmplitudes4", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4),
#     [-0.6091325 ,  0.9843762 ,  0.4280502 ,  0.05832202, -0.35152913,
#         1.93139409,  0.86961548, -0.33681776,  0.51244531, -0.68913959,
#         1.61590261,  1.46769233,  0.49653955,  0.00337373, -0.22153556,
#         0.65560437, -0.79022439,  0.17265976, -0.19782632,  0.13520074]),
#     ("SPSA200-PauliFeature4-RealAmplitudes4", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4),
#     [ 1.15866768,  0.70848972,  1.78028578,  1.73137049,  0.42013982,
#         0.61562094, -1.12609087,  2.43094607, -0.74089255,  0.11270528,
#        -0.71075179,  0.29445935, -0.57013928, -0.43977206,  1.32181862,
#        -0.32971983, -0.79119886,  0.59496394,  0.68789662,  1.45898654])
#     ("COBYLA200-PauliFeature4-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2),
#     [ 0.67576386,  1.17025501,  2.29751264,  1.00526572,  0.73093493,
#        -0.32573605,  3.08669819,  0.47565819,  0.746639  ,  2.05322059,
#         1.76045157, -0.01179355]),
#     ("SLSQP200-PauliFeature4-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2),
#     [-4.20617495,  4.24227151,  3.41205716, -2.54771031, -3.43895721,
#         1.6872467 , -0.76319247, -5.58470957,  4.92476986,  0.42090465,
#         3.29151146, -3.10355457]),
#     ("SLSQP200-PauliFeature4-RealAmplitudes4", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4),
#     [ 1.4191829 ,  3.64509514,  2.20004535, -5.20315387,  3.08036269,
#         0.17483316,  1.91951741,  3.48743157,  0.48693566, -0.62908158,
#         0.40281044,  1.83764268,  3.34391617,  1.25884651,  2.42287698,
#         1.32127008,  1.4792624 ,  1.67027256,  1.67804439, -1.39477913]),
#     ("COBYLA200-ZFeature4-RealAmplitudes4", ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4),
#     [ 1.96652569,  0.24217319,  1.47837951,  0.83915742,  2.29978635,
#         0.69589474,  0.24225658,  0.09114969,  0.6943839 ,  1.70546927,
#         0.0490381 ,  1.89544691,  0.85960574,  1.35149849, -0.3217444 ,
#        -0.31325965,  0.39365852,  0.60165538,  0.70124588,  0.53969418]),
#     ("SLSQP200-ZZFeature1-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4),
#     [ 0.11349474,  1.0334961 , -0.04275785, -0.25492248,  1.1334802 ,
#        -0.21303143, -0.09640077, -0.64781828,  3.03666383, -0.26649666,
#         1.29083205, -0.84421272, -1.37699883,  2.12816912,  0.35291702,
#         3.24184637, -0.33454144,  1.50397921,  0.88973354, -0.46883434]),
#     ("SLSQP200-PauliFeature2-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2),
#     [ 0.0931631 ,  1.35448865,  1.30810369,  1.27211711, -0.85509695,
#         1.54229113,  0.97813028,  0.16678624, -0.09652245, -1.34880985,
#         0.09515564,  2.10471579]),
#       ("SLSQP200-PauliFeature2-RealAmplitudes4", PauliFeatureMap(num_inputs, reps=2, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), 
#        [ 4.80888499,  0.14485586,  1.18760828, -1.95478935, -1.95636456,
#        -1.68479935,  0.25224422,  1.23705719, -0.97159198, -0.47808974,
#        -3.59987975,  4.14020304, -0.34507723,  1.83050263, -1.84984624,
#         1.3138018 ,  3.74109934,  2.03769869,  2.51983469, -1.4500285 ]),
#     ("SLSQP200-PauliFeature4-RealAmplitudes1", PauliFeatureMap(num_inputs, reps=4, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), 
#        [-0.06500449,  2.17072138,  2.22459104, -1.05176998, -0.30855382,
#         0.60190422,  0.62994114,  0.57937807]),
    
    # Trained on symulator 120 iterations
    
    # Trained on real devices
    ("COBYLA50-PauliFeature1-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), 
       [ 0.21510057,  0.72720136,  0.61561149,  0.18645492,  0.12018683,  2.26401866,
  1.8264945,   1.83291486,  0.26298326,  0.07541654,  0.27751341, -0.20378999 ]),
    ("COBYLA120-ZZFeature1-RealAmplitudes4", ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), 
       [ 0.62717535,  0.34029388,  2.57974013,  2.17992943,  0.83700215,  0.59067521,
 -0.61045692,  1.95150747, -0.46986697,  0.13879197,  0.21185469,  1.73558887,
 -0.20386651, -0.38735644,  0.48317553, -0.65251228,  0.03862765,  0.11063922,
  0.38836711, -0.09068928 ]),
    ("SPSA120-ZFeature4-RealAmplitudes2", ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), 
       [ 0.75978053, -1.68554078,  0.92478598,  1.14276253,  0.57254153,  0.05458684,
  0.47247588, -0.20933354,  0.96685274,  0.12331087, -1.09344216, -1.186017 ]),
    ("SPSA120-ZFeature4-RealAmplitudes4", ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), 
       [ 0.92039765, -1.65728503,  1.37365554,  2.00217866,  1.26155323,  1.71472258,
  0.99318311,  3.52031406, -0.33064895, -0.86852314,  0.98825511, -0.11277225,
  1.02111117, -1.66502468, -1.48629686,  0.677785,   -0.26212068,  1.31246648,
  0.94127439,  1.50654558 ]),
    ("SPSA75-ZZFeature2-RealAmplitudes1", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), 
       [ 1.48673248,  2.01602537, -0.84008807, 2.34967427, -0.22776806,  2.91573314,
 -0.63474715,  1.56521638 ]),
    ("SPSA120-PauliFeature1-RealAmplitudes1", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), 
       [ -1.39409237,  1.37084711,  0.04601383,  0.29830453,  1.45454256,  1.04155995,
  1.7957715,   1.50398782 ]),
    ("SPSA120-PauliFeature1-RealAmplitudes4", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), 
       [ 1.15535312, -0.33569659,  1.20364857,  2.22316088,  0.06201333,  2.37818058,
  0.27315897,  2.19511958, 4.75550936,  1.36695625, -0.51180575, -0.14281792,
  1.7033721,   0.93027766,  0.00858066,  1.83045975,  0.67256963, -1.35715258,
  1.72969962, -3.74312407 ]),
    ("SLSQP120-PauliFeature1-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), 
       [ 0.14959474,  0.77453575,  0.29588322, -0.30826619, -0.42511299,  0.00842944,
  0.19853805, -0.02982124,  2.46286779,  0.1610717,   0.23648886,  1.73409717 ]),
    
    ("COBYLA120-PauliFeature1-RealAmplitudes2", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), 
       [ 1.46095036,  0.33959095, -0.21365186,  1.18775194,  1.14046309,  2.10422988,
  2.24324231,  1.33655136, -0.09158358, -0.24403268,  0.07191387,  0.01557951 ]),
    ("SPSA120-ZZFeature2-RealAmplitudes1", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), 
       [ -0.4469402,  -1.59476924, -0.76933623,  0.86009125,  0.30644173,  0.05032752,
 -0.02815303, -0.28299178 ]),
    
    # Symulator training after 120 iterations
        ("COBYLA120-PauliFeature1-RealAmplitudes2-sym", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), 
       [ 1.18870768,  0.7419696 ,  0.56850968, -0.77341035,  0.32570143,
        1.67877559,  1.47539948,  2.22569714,  1.02144073,  1.7108039 ,
        0.99566847, -0.52297682 ]),
    ("COBYLA120-ZZFeature1-RealAmplitudes4-sym", ZZFeatureMap(num_inputs, reps=1), RealAmplitudes(num_inputs, reps=4), 
       [ 0.85616783,  0.70317059,  0.76061866,  0.43061746,  0.96628795,
        0.51346737,  0.79387614,  1.03633317,  2.26312407, -0.32100554,
        1.02302765,  0.82466267,  0.45859887,  0.47735029,  0.29134625,
        1.04199089, -0.36926641, -0.33385468, -0.47271232,  0.28590156 ]),
    ("SPSA120-ZFeature4-RealAmplitudes2-sym", ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=2), 
       [ 0.57094267,  1.06686838,  1.14670198,  0.6986171 ,  0.91636786,
       -0.26440811,  0.48704445,  0.44202094,  1.26665897,  0.74015288,
        0.1844098 ,  1.35705913 ]),
    ("SPSA120-ZFeature4-RealAmplitudes4-sym", ZFeatureMap(num_inputs, reps=4), RealAmplitudes(num_inputs, reps=4), 
       [ 1.56770877,  1.52457098,  0.59533622, -1.46643708, -1.09689963,
        1.28468155, -0.15583574,  1.36165806,  1.83805353,  0.13506075,
        1.97939682,  0.071616  ,  0.16186114, -0.29875618,  0.44939049,
        0.83674555, -1.8503622 ,  1.98816026, -0.67138079, -0.5359867 ]),
    ("SPSA120-ZZFeature2-RealAmplitudes1-sym", ZZFeatureMap(num_inputs, reps=2), RealAmplitudes(num_inputs, reps=1), 
       [ -0.00868876,  1.48747628, -1.35063657,  0.2071475 , -0.07329166,
        1.65232211,  0.79862956,  2.95815133 ]),
    ("SPSA120-PauliFeature1-RealAmplitudes1-sym", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=1), 
       [ 1.69337818,  1.0211933 , -0.04366962, -0.80919879,  1.09468878,
        1.57562664, -0.56098705,  2.03929672 ]),
    ("SPSA120-PauliFeature1-RealAmplitudes4-sym", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=4), 
       [ -0.717922  ,  0.84417176,  0.5919755 ,  0.18733031, -0.25324822,
        2.04933748,  0.84845263, -0.47226339,  0.30206645, -0.52870456,
        1.48163888,  1.29987785,  0.3459425 ,  0.18359147, -0.32270978,
        0.51755933, -0.95182375,  0.04240512, -0.36465026,  0.0128467 ]),
    ("SLSQP120-PauliFeature1-RealAmplitudes2-sym", PauliFeatureMap(num_inputs, reps=1, paulis= ['Z', 'Y', 'ZZ']), RealAmplitudes(num_inputs, reps=2), 
       [ 1.41806476,  0.87030179,  0.25658058,  3.0987385 ,  0.25125016,
        1.17820206,  0.53310684,  2.17380386,  1.5228631 , -0.46230242,
        0.20157721,  1.09280113 ]),
]
scores = []
for case in test_cases:
    circuit = construct_circuit(case[1], case[2], quantum_instance, num_inputs)
    score = run_and_score(circuit, X_test, Y_test, case[3])
    scores.append([case[0], score])

[['COBYLA50-PauliFeature1-RealAmplitudes2', 0.5333333333333333],
 ['COBYLA120-ZZFeature1-RealAmplitudes4', 0.35555555555555557],
 ['SPSA120-ZFeature4-RealAmplitudes2', 0.7777777777777778],
 ['SPSA120-ZFeature4-RealAmplitudes4', 0.35555555555555557],
 ['SPSA75-ZZFeature2-RealAmplitudes1', 0.3333333333333333],
 ['SPSA120-PauliFeature1-RealAmplitudes1', 0.6222222222222222],
 ['SPSA120-PauliFeature1-RealAmplitudes4', 0.3333333333333333],
 ['SLSQP120-PauliFeature1-RealAmplitudes2', 0.35555555555555557],
 ['COBYLA120-PauliFeature1-RealAmplitudes2', 0.35555555555555557],
 ['SPSA120-ZZFeature2-RealAmplitudes1', 0.35555555555555557],
 ['COBYLA120-PauliFeature1-RealAmplitudes2-sym', 0.24444444444444444],
 ['COBYLA120-ZZFeature1-RealAmplitudes4-sym', 0.3333333333333333],
 ['SPSA120-ZFeature4-RealAmplitudes2-sym', 0.5555555555555556],
 ['SPSA120-ZFeature4-RealAmplitudes4-sym', 0.4666666666666667],
 ['SPSA120-ZZFeature2-RealAmplitudes1-sym', 0.37777777777777777],
 ['SPSA120-PauliFeature1-RealAmplit

In [18]:
scores

[['COBYLA50-PauliFeature1-RealAmplitudes2', 0.5111111111111111],
 ['COBYLA120-ZZFeature1-RealAmplitudes4', 0.5333333333333333],
 ['SPSA120-ZFeature4-RealAmplitudes2', 0.7111111111111111],
 ['SPSA120-ZFeature4-RealAmplitudes4', 0.5555555555555556],
 ['SPSA75-ZZFeature2-RealAmplitudes1', 0.35555555555555557],
 ['SPSA120-PauliFeature1-RealAmplitudes1', 0.8666666666666667],
 ['SPSA120-PauliFeature1-RealAmplitudes4', 0.3111111111111111],
 ['SLSQP120-PauliFeature1-RealAmplitudes2', 0.7555555555555555]]